In [ ]:
from __future__ import division
import numpy as np
from astropy.table import Table
%matplotlib inline  
import matplotlib.pyplot as plt
from ipywidgets import interactive, interact

In [ ]:
gases = Table.read('ipcc-gas-concentrations.csv', format='ascii.ecsv')

In [ ]:
temps = Table.read('graph.csv')

In [ ]:
anth_emit = Table.read('anthro_emissions.ecsv', format='ascii.ecsv')

In [ ]:
land_use = Table.read('Global_land-use_flux-1850_2005.csv')

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(gases['Year'], gases['CO2'])
plt.ylabel('CO2 concentration, ppm')
plt.grid()
#plt.xlim(1850, 1900)
#plt.ylim(285, 300)

## Human-related CO$_2$

First we need to convert from ppm to Gto (gigatons). From *An Introductory Global CO2 Model: (with Companion Media Pack)*, p. 30,

$$
    ppm = \frac{GtC/MWc}{GtA/MWa}10^6,
$$

where $GtC$ is the number of gigtons of carbon, $MW$ is molecular weight (12 for carbon, 29 for air) and $GtA$ is the mass of the air in gigtons. Note that $1 Gt = 10^{12}$kg.

Rearranging,

$$
GtC = GtA\frac{MWc}{MWa}\frac{ppm}{10^6}.
$$

[Wikipedia says](https://en.wikipedia.org/wiki/Atmosphere_of_Earth#Density_and_mass) that the mass of the atmosphere is $5.184\times 10^{18}$kg, which is $5.184\times 10^6$Gt.

In [ ]:
def ppm_to_gtc(ppm):
    """
    Return amount of carbon in Gt from concetration in parts per million.
    """
    mw_carbon = 12
    mw_air = 29
    mass_air = 5.184e6 # Gt
    return mass_air * ppm / 1e6 * mw_carbon / mw_air

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(gases['Year'], ppm_to_gtc(gases['CO2']))
plt.ylabel('Amount of Carbon, GtC')
plt.xlabel('Year')
plt.xlim(1850, 2011)
plt.grid()

## now calculate a rate....

Let's do a quick and dirty estimate....get a rate by doing a crappy numerical derivative, assume all of it is anthroppgenic.

In [ ]:
plt.figure(figsize=(10, 5))

human_co2 = ppm_to_gtc(gases['CO2'])

human_rate = human_co2[1:] - human_co2[:-1]
plt.plot(gases['Year'][1:], human_rate)
plt.ylabel('Anthropogenic CO2 emissions, GtC/yr')

## Well, sort of...

The numbers after 1950 don't look too bad, but earlier are purely noise. The vertical scale is about the right size.

# Need to find better source for anthropogenic rate

Done! Dig far enough down in the IPCC report and you get to Working Group 1 Technical Summary Figure TS.4, which, in its caption, explains that the data comes from the [Carbon Dioxide Information Analysis Center](http://cdiac.ess-dive.lbl.gov/), which has a downloadable table of anthropogenic emissions at http://cdiac.ess-dive.lbl.gov/ftp/ndp030/global.1751_2014.ems

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(anth_emit['Year'], anth_emit['Total']/1e3 * 3.667, label='Fossil fuels/cement')
plt.plot(land_use['Year'], land_use['Global']/1e3 * 3.667, label='Land use')
plt.title('CO2 emissions from coal, gas, oil')
plt.legend()
plt.grid()
plt.xlabel('Year')
plt.ylabel('CO2 emissions, GtCO2/year')
#plt.xlim(1880, 2015)

## Sanity check: does integrated emissions plus concetration in 1750 match observations?

### NOTE: GtC is NOT the same as GtCO2....

The latter is the former times 3.667 according to CDIAC

In [ ]:
cumulative_emissions = np.cumsum(anth_emit['Total']/1e3)
cumulative_land_use = np.cumsum(land_use['Global']/1e3)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(1750, 2014), cumulative_emissions + human_co2[0], label='integrated')
plt.plot(land_use['Year'], cumulative_land_use + human_co2[0]+ 36, label='land use integrated')
plt.plot(gases['Year'], human_co2, label='observed')
plt.legend()
plt.grid()

### No match...

At least part of that is because not all emitted CO2 stays in the atmosphere. Some is absorbed by the oceans, some by the ground.

Increases prior to 1880 were primarily from land use changes, I think.

# Temperature plots

## Measured temperature

All of the temperatures below are relative to the 1950-1980 mean temperature, which is, from [Hansen et al.: Global Surface Temperature Change](https://pubs.giss.nasa.gov/docs/2010/2010_Hansen_ha00510u.pdf), roughly 14$^\circ$C, or 287K, with an uncertainty of "Several tenths of a degree C."

In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(temps['Year'], temps['No_Smoothing'])
plt.ylabel('Global temperature anomoly (C)')
plt.xlabel('Year')

# The model

This model is based on that presented in the last section of Unit T of [Six Ideas that Shaped Physics](http://www.physics.pomona.edu/sixideas/), an excellent introductory physics text. The climate change model is described in these [slides from a talk at the winter 2016 AAPT meeting](http://www.physics.pomona.edu/sixideas/assets/aaptw16.pdf) by the author, Thomas Moore.

The model in this notebook differs from that in the talk/textbook in a few important ways, which are noted below.

The model is deliberately very simple, ignoring a large number of factors which obviously affect the climate.

Essentially, the atmosphere is modeled as one or more thin shells which absorb (and then re-emit) the radiation from Earth's surface. The model is developed by imagining first one perfectly-absorbing shell, then two, then three, then generalize to $n$ shells. Once we have an expression for $n$ shells, we assume the model works for fractional values of $n$.

## Radiative balance

We begin by assuming the earth and atmosphere are in equilibrium, with the radiation from the top of the atmosphere (TOA in the climate change literature) balanced by the incoming solar radiation. 

We begin with an Earth that has no atmosphere. If we assume that the Earth receives power from the sun $P_{sun}$ and radiates all of that back to space at temperature $T_{bare}$, then we can determine that temperature from

$$
P_{sun} = A \sigma T_{bare}^4,
$$

where $\sigma=5.67\times 10^{-8}$W/m$^2$/K$^4$ is the Stefan-Boltzman constant. We assume the Earth radiates in all directions, so the area $A$ in this expression is $4\pi R_{E}^2$. Moore uses as the flux from the Sun $F_{sun, true} = 960$W/m$^2$ (in roughly agreement with [Kiehl and Trenberth 1997](http://journals.ametsoc.org/doi/pdf/10.1175/1520-0477%281997%29078%3C0197%3AEAGMEB%3E2.0.CO%3B2) who, in their last figure, have 235 W/m$^2$ as the total for radiation absorbed by the Earth and atmosphere; this is 960/4, so the incoming radiation averaged over the surface of the Earth). If we make the simplifying assumption that the Earth is a flat disk that faces the Sun (i.e. area $\pi R_{E}^2$) and solve for $T_{bare}$ we get

$$
T_{bare} = \left(\frac{P_{sun}}{A\sigma}\right)^{1/4} = \left(\frac{\pi R_E^2F_{sun,true}}{4\pi R_E^2\sigma}\right)^{1/4}
= \left(\frac{F_{sun, true}}{4 \sigma}\right)^{1/4} = 255\text{K}.
$$

The combination $F_{sun, true}/4$ will be called $F_{sun}$ for the remainder of this.

## Single layer, perfectly absorbing atmosphere 

If we now assume that the atmosphere is a single thin layer that is perfectly transparent to solar radiation but opaque to radiation from the surface of the Earth, then the only change is that this layer has twice the surface area as the earth, because it can radiate both up (into space) and down (towards the Earth). 

The assumption of equilibrium requires that the radiation up equal the incoming solar flux. That implies the the flux hitting the surface of the Earth doubles: in addition to the flux coming directly from the sun, the same flux must come from the atmosphere. That means that 

$$
T_1 = \left(\frac{2F_{sun}}{\sigma}\right)^{1/4} = 2^{1/4} T_{bare}.
$$

## More layers

If we add a second layer to this atmosphere, then the top layer must radiate $F_{sun}$ outwards (to balance the incomign flux from the Sun), and so it radiates the same flux downards towards the layer below it. The top layer has incident radiation only from below, so the layer below must be radiating $2F_{sun}$ upwards towards the top layer. That means the bottom layer emits a total of $4F_{sun}$, two up towards the top layer, two down towards the surface of the Earth.

The Earth therefore is receiving $3F_{sun}$; one from the Sun itself, two from the layer above it, and its temperature must be

$$
T_2 = \left(\frac{3F_{sun}}{\sigma}\right)^{1/4} = 3^{1/4} T_{bare}.
$$

If we add a third layer to the model, then the top two layers behave as above. The bottomost layer must be radiating $3F_{sun}$ upwards to provide the energy for the layer above, because that layer above radiates $4F_{sun}$ but only gets $F_{sun}$ from the top layer. If the bottommost layer radiates $3F_{sun}$ upwards it does the same thing down, so the Earth is receiving $4 F_{sun}$; one from the Sun and three from the atmosphere. Its temperature is

$$
T_3 = \left(\frac{4F_{sun}}{\sigma}\right)^{1/4} = 4^{1/4} T_{bare}.
$$

### $n$ layers

The trend by now ought to be clear. For $n$ layers, the Earth receives flux $(n+1)F_{sun}$, one from the Sun and the rest from the bottommost layer. The temperature in this case

$$
T_n = \left(\frac{(n+1)F_{sun}}{\sigma}\right)^{1/4} = (n+1)^{1/4} T_{bare}.
$$

## If it works for integers, assume it works for real numbers

In one last leap, we assume that the result for $n$ layers works even if $n$ is not a whole number. It will turn out that for Earth, $n$ is smaller than 1.

## Connect the model to reality

The goal is to see if we can work up a simple model that relates concentration of CO$_2$ in the atmosphere to surface temperature. Doing so requires that we work out how $n$ is related to carbon dioxide concetration. As a first step in that we need to recognize that most of the absorption (and hence re-emission) in the atmosphere is due to gases other than carbon dioxide (primarily water vapor).

We assume that we can write $n$ as a sum of contributions from different gases,

$$
n = n_{CO_{2}} + n_{water} + n_{methane} + \cdots.
$$

For simplicity we break this down into just two pieces, carbon dioxide and everything else

$$
n = n_{CO_2} + n_{other}.
$$

In the climate change literature, what we are calling $n$ is proportional to radiative forcing, the radiative flux due to a particular comoponent of the climate model. Those fluxes are routinely (and quite sensibly) added, so adding contributions to $n$ seems appropriate.

The relationship  between $n_{CO_2}$ and the concentration of carbon dioxide in atmosphere is $n\propto \rho^{1/2}$, where $\rho$ is the concentration. Moore cites this [Science of Doom page](https://scienceofdoom.com/2010/02/05/co2-an-insignificant-trace-gas-part-four/) which uses results from band models of carbon dioxide absorption to reach this relationship, assuming that the limit of strong absorption.
If we know for some carbon dioxide concentration $\rho_{ref}$ that the corresponding value of $n_{CO_2}$ is $n_{CO_2, ref}$, then for any other concentration we have 
$$
n_{CO_2} = \left(\frac{\rho}{\rho_{ref}}\right)^{1/2} n_{CO_2, ref}.
$$
If this relationship is badly off, we ought to see it in the final model. 

We still need to determine how to apportion $n$ between carbon dioxide and other gasses. Assume that at some point that we denote by subscript $b$ that we know the fraction $f$ of absoprtion due to carbon dioxide. Then at that time $n_{CO_2,b} = f n_b$ and we can write

$$
n_b = n_{CO_2,b} + (1-f)n_b.
$$

If we assume that the other absorption stays constant with time then at any time $n$ is given by

$$
n = \left(\frac{\rho}{\rho_{b}}\right)^{1/2} n_{CO_2, b} + (1-f)n_b 
= f n_b \left(\left(\frac{\rho}{\rho_{b}}\right)^{1/2} + \frac{1}{f} - 1\right)
$$

To recap, we need to know what fraction $f$ of the absorption $n_b$ at this "base" time is due to carbon dioxide. If we know that (and $n_b$) then we can calculate $n_{CO_2}$ at that time and use the concentration scaling to calculate $n$ at some other time. 

If we also know the temperature $T_b$ at the "base" time then we can calculate $n_b$ from the temperature relationship  in the previous section, since

$$
n_b = \left(\frac{T_b}{T_{bare}}\right)^4 - 1.
$$
It is possible, if desired, to calibrate $n$ at some other point as long as we know the carbon dioxide concentration and temperature at that point.

We can combine the relationships for $n$ and $n_b$ to obtain the model we want that relates carbon dioxide concentration to temperature,

$$
T = (n+1)^{1/4} T_{bare} = \left[f\left(\left(\frac{T_b}{T_{bare}}\right)^4 - 1\right) \left(\left(\frac{\rho}{\rho_{b}}\right)^{1/2} + \frac{1}{f} - 1\right) + 1\right]^{1/4} T_{bare}
$$

This is admittedly a little messy-looking, but boils down to 

$$
T = \left(A\rho^{1/2} + B\right)^{1/4} T_{bare},
$$

if we define 

$$
A = \frac{f}{\rho_b^{1/2}}\left(\left(\frac{T_b}{T_{bare}}\right)^4 - 1\right) = \frac{fn_b}{\rho_b^{1/2}}
$$

and

$$
B = f + (1-f)\left(\frac{T_b}{T_{bare}}\right)^4 = f +(1-f)(1+n_b)
$$


## Numerical value of the parameters

Fortunately, [Kiehl and Trenberth 1997](http://journals.ametsoc.org/doi/pdf/10.1175/1520-0477%281997%29078%3C0197%3AEAGMEB%3E2.0.CO%3B2) work out the fraction $f$ of absoprtion due to carbon dioxide and find it to be 27% (actually 26% with no clouds and 28% with clouds). They clearly state that they modeled an atmosphere with a CO$_2$ concentration of 353ppm, the value in given by the IPCC in its 1990 report. 

Furthermore, the measured temperature at that time was about 0.35K above the 1950-1980 baseline, so was 287.35K, from which we can determine $n_b$ to be 0.610 (which differs from Moore's value of 0.605, more on that later). Using $T_{bare} = 255$K the numerical values of the parameters $A$ and $B$ are $A=8.77\times 10^{-3}$ppm$^{-1/2}$ and $B=1.45$.

In [ ]:
n_b = (287.25/255)**4 -1
n_b

## Comparison to Moore

Calibrating this scaling is the  place that I think Moore goes awry. He assumes that 27% of the absorption in the pre-industrial era, when the carbon dioxide concentration was 280 ppm, is due to carbon dioxide, citing a result from the [Science of Doom site](https://scienceofdoom.com/2010/02/10/co2-%E2%80%93-an-insignificant-trace-gas-part-five/) (which really does a nice job of discussing the materials). However, that site refers to the paper by [Kiehl and Trenberth 1997](http://journals.ametsoc.org/doi/pdf/10.1175/1520-0477%281997%29078%3C0197%3AEAGMEB%3E2.0.CO%3B2), which clearly states that they modeled an atmosphere with a CO$_2$ concetration of 353ppm, the value in given by the IPCC in its 1990 report. 

Moore assumes that the carbon dioxide concentration in the pre-industrial era is 280 ppm, so Moore's calculation of $n$ effectively uses the wrong value for $\rho_b$. 

However, Moore makes a second mistake. He calculates $n_b$ by using the pre-industrial temperature 287K, which he asserts was the temperature in 1900, and that the concentration at that time was 280 ppm. In fact, from the data above, the temperature in 1900 was 286.8 (temperature anomaly of -0.2), and at that temperature the carbon dioxide concentration was 296 ppm, not 280 ppm.

The temperature mistake is not that serious; it can be accomodated when we fit the model to the data by adding a fixed temperature offset. Given the "several tenths of a degree C" in the average temperature 1950-1980 that is probably not really an issue.

In terms of the model above, Moore used $f=0.27$, $T_b = 287$K, and $\rho_b = 280$ppm. Using these numbers I calculate values for $A$ and $B$ of $9.76\times 10^{-3}$ ppm$^{-1/2}$ and $B=1.44$.


## Looks good, code up the prediction

In [ ]:
def temp_model(inp_time, concetration, lag=0, temp_bare=255, 
               n_other=0.442, n_co2_base=0.163, ppm_base=296):
    out_time = inp_time + lag
    n_tot = n_other + n_co2_base * np.sqrt(concetration / ppm_base)
    out_temp = (1 + n_tot) ** 0.25 * temp_bare
    return (out_time, out_temp)
    

In [ ]:
def temp_model2(inp_time, concentration, temp_bare=255, f=0.27, Tb=287.35, ppm_b=353):
    n_b = (Tb/temp_bare)**4 -1
    A = f * n_b / ppm_b**0.5
    B = f + (1 - f) * (1 + n_b)
    out_temp = (A * np.sqrt(concentration) + B)**0.25 * temp_bare
    return (inp_time, out_temp)


In [ ]:
print(temp_model(0, 400))
print(temp_model2(0, 400))

In [ ]:
mod_time, mod_temp = temp_model2(gases['Year'], gases['CO2'] - 16, Tb=287.0, ppm_b=280)
mod_time2, mod_temp2 = temp_model2(gases['Year'], gases['CO2'], f=0.27, Tb=287.35, ppm_b=353)

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(mod_time, mod_temp, label='Moore')
plt.plot(mod_time2, mod_temp2, label='Me')
plt.legend()
plt.grid()

In [ ]:

@interact(temp_offset=(0, 0.5, 0.01), lag=(0, 30, 1))
def plot_model(lag=30, temp_offset=0.25):
    plt.figure(figsize=(10, 5))

    plt.plot(temps['Year'], temps['No_Smoothing'] + 0.25, label='observed')
    #plt.plot(temps['Year'], temps['Lowess(5)'], label='obs, smoothed')
    mod_year = mod_time + lag
    in_ref = (np.array(mod_year) > 1950) & (np.array(mod_year) < 1980)
    avg_ref = np.mean(mod_temp[in_ref])
    plt.plot(mod_time + lag, mod_temp - 287 - temp_offset, label='model', linewidth=3)
    plt.plot(mod_time2 + lag, mod_temp2 - 287 + 0.25, label='model new', linewidth=3)

    plt.ylabel('Global temperature anomoly (C)')
    plt.xlabel('Year')
    plt.legend()
    plt.xlim(1750, 2050)
    plt.grid()

In [ ]:
@interact(f_b=(0, 1, 0.005), Tb=(286.5, 287.5, 0.05), ppm_b=(280, 355, 1), lag=(0, 30, 1))
def plot_model(f_b, Tb, ppm_b, lag=0):
    plt.figure(figsize=(10, 5))
    plt.plot(temps['Year'], temps['No_Smoothing'] + 287, label='observed')

    moore_time , moore_temp = temp_model2(gases['Year'], gases['CO2'] - 16, f=0.27, Tb=287, ppm_b=280)
    #plt.plot(moore_time, moore_temp, label='Moore')
    mod_time3, mod_temp3 = temp_model2(gases['Year'], gases['CO2'], f=f_b, Tb=Tb, ppm_b=ppm_b)
    mod_year = mod_time 
    in_ref = (np.array(mod_year) > 1950) & (np.array(mod_year) < 1980)
    avg_ref = np.mean(mod_temp[in_ref])
    plt.plot(mod_time3 + lag, mod_temp3, label='Model', linewidth=3)

    plt.ylabel('Global temperature anomoly (C)')
    plt.xlabel('Year')
    plt.legend()
    plt.xlim(1750, 2050)
    plt.ylim(286, 288.5)
    plt.grid()

In [ ]:
@interact(f=(0, 1, 0.01), Tb=(286.5, 287, 0.1), ppm_b=(280, 350, 1))
def plot_model3(f, Tb, ppm_b):
    plt.figure(figsize=(10, 5))
    #plt.plot(temps['Year'][:-2], temps['No_Smoothing'][:-2] + 287.25, label='observed')

    moore_time , moore_temp = temp_model2(gases['Year'], gases['CO2']-16, f=0.27, Tb=287, ppm_b=280)
    plt.plot(gases['CO2'], moore_temp, label='Moore')
    mod_time3, mod_temp3 = temp_model2(gases['Year'], gases['CO2'], f=f, Tb=Tb, ppm_b=ppm_b)
    mod_year = mod_time 
    in_ref = (np.array(mod_year) > 1950) & (np.array(mod_year) < 1980)
    avg_ref = np.mean(mod_temp[in_ref])
    plt.plot(gases['CO2'], mod_temp3, label='model new', linewidth=3)

    plt.ylabel('Global temperature anomoly (C)')
    plt.xlabel('carbon dioxide concentration (ppm)')
    plt.legend()
    #plt.xlim(1750, 2050)
    plt.ylim(285, 291)
    plt.grid()